## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.
3. pinecone api 키가 필요합니다. api발급 방법은 https://app.pinecone.io/ 에서 확인 해주세요. 그리고 테스트 이후 pinecone 인덱스를 반드시 삭제해야 합니다.
4. milvus서버가 떠 있어야 정상 동작합니다. ./start_milvus.sh 를 실행하여 milvus 서버를 띄워주세요.

## 목적:
- Pinecone, Milvus, Qdrant 세 가지 벡터 데이터베이스에 동일한 벡터를 삽입
- 여러 벡터 검색 시스템을 비교하고 통합할 때 유용함

## 왜 필요한가?
- 서로 다른 벡터 데이터베이스의 성능을 비교할 수 있음
- 특정 벡터 검색 시스템을 선택할 때 도움이 됨

## 주요 개념:
- **Pinecone**: 클라우드 기반 벡터 검색 서비스, 빠른 검색 및 확장성 제공
- **Milvus**: 대용량 벡터 검색을 지원하는 오픈소스 벡터 DB
- **Qdrant**: 속도와 성능을 고려한 오픈소스 벡터 검색 데이터베이스

In [1]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
import os
from pinecone import Pinecone, ServerlessSpec
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
# 환경 변수에서 Pinecone API 키 가져오기
pinecone_api_key = os.environ.get("PINECONE_API_KEY", "your_api_key")

# Pinecone 설정
pc = Pinecone(api_key=pinecone_api_key)
index_name = "ex4-sample-index"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=384,
        metric='euclidean',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
pinecone_index = pc.Index(index_name)

In [4]:
# Qdrant 연결
qdrant_client = QdrantClient(host="localhost", port=6333)
collection_name = "ex4_sample_vectors"

# 🚀 Qdrant 컬렉션 생성 (없을 경우 자동 생성)
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384는 임베딩 차원 수
    )

In [5]:
# Milvus 연결 및 컬렉션 생성
connections.connect("default", host="localhost", port="19530")
milvus_schema = CollectionSchema(fields=[
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),  # auto_id 활성화
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=384)
])
milvus_collection = Collection(name=collection_name, schema=milvus_schema)

In [6]:
# LangChain Sentence Transformers 사용
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding_vector = embeddings.embed_query("Sample text")

if embedding_vector is None:
    raise ValueError("오류: 생성된 임베딩 벡터가 None입니다. 모델이 올바르게 로드되었는지 확인하세요.")

In [7]:
# Qdrant에 삽입
qdrant_client.upsert(
    collection_name=collection_name,
    points=[models.PointStruct(id=1, vector=embedding_vector)]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
# Pinecone에 삽입
pinecone_index.upsert(vectors=[("id1", embedding_vector)])

{'upserted_count': 1}

In [9]:
# Milvus에 삽입
milvus_collection.insert([[embedding_vector]])

print("Qdrant, Pinecone, Milvus에 벡터 저장 완료!")

Qdrant, Pinecone, Milvus에 벡터 저장 완료!


In [10]:
# 테스트 완료 후 인덱스 삭제
def cleanup():
    # Qdrant 컬렉션 삭제
    qdrant_client.delete_collection(collection_name=collection_name)
    print("Qdrant 컬렉션 삭제 완료")

    # Pinecone 인덱스 삭제
    pc.delete_index(index_name)
    print("Pinecone 인덱스 삭제 완료")

    # Milvus 컬렉션 삭제
    milvus_collection.drop()
    print("Milvus 컬렉션 삭제 완료")

# 정리 실행
cleanup()



Qdrant 컬렉션 삭제 완료
Pinecone 인덱스 삭제 완료
Milvus 컬렉션 삭제 완료
